In [195]:
import os
import sys
import numpy as np
import pandas as pd
import pandas_datareader as pdr

# Hack to ensure the notebook can load local modules by appending the parent directory to the path
# Ensure a '.env' file is in the workspace root
from dotenv import find_dotenv
sys.path.append(os.path.dirname(find_dotenv()))
from alphasim.backtest import backtest

In [196]:
price_df = pdr.get_data_yahoo(['VTI', 'TLT'])
price_df = price_df['Adj Close']

display(price_df)

Symbols,VTI,TLT
Date,,
2017-10-09,120.264015,111.911880
2017-10-10,120.548790,112.092445
2017-10-11,120.714119,112.309052
2017-10-12,120.567154,112.787498
2017-10-13,120.658997,113.599838
...,...,...
2022-09-30,179.470001,102.206001
2022-10-03,184.029999,103.830002
2022-10-04,189.940002,103.540001


In [197]:
weight_df = price_df.copy()
weight_df['VTI'] = 0.6
weight_df['TLT'] = 0.4

display(weight_df)

Symbols,VTI,TLT
Date,,
2017-10-09,0.6,0.4
2017-10-10,0.6,0.4
2017-10-11,0.6,0.4
2017-10-12,0.6,0.4
2017-10-13,0.6,0.4
...,...,...
2022-09-30,0.6,0.4
2022-10-03,0.6,0.4
2022-10-04,0.6,0.4


In [198]:
def min_commission(trade_price, trade_size):
    return 10

In [199]:
max_lev = 1
vola_target = 0.10
trade_buffer = 0.05
initial_cap = 10000.0
result = backtest(price_df, weight_df, min_commission, trade_buffer, initial_cap)

In [200]:
assert(price_df.shape == weight_df.shape)

price_df['cash'] = 1
weight_df['cash'] = max_lev - weight_df.sum(axis=1)

pf_df = pd.DataFrame(index=price_df.index, columns=weight_df.columns)
pf_df[:] = 0.0

pnl_df = pf_df.copy()


'''
current_weight = (size * price) / nav
target_weight = weight
delta_weight = target_weight - current_weight
do_trade = delta_weight > trade_buffer

In fixed_pct_comm scheme:
trade_weight = target_weight - trade_buffer

In min_comm scheme:
trade_weight = target_weight

trade_value = trade_weight * nav
trade_size = trade_value / price
'''

display(pf_df.iloc[0])

Symbols
VTI     0.0
TLT     0.0
cash    0.0
Name: 2017-10-09 00:00:00, dtype: object

In [205]:
n = len(price_df)

result = pd.DataFrame()

for i in range(n):

    pf = None
    if i == 0:
        pf = pf_df.iloc[0]
        pf['cash'] = initial_cap
    else:
        pf = pf_df.iloc[i-1]

    # Slice data to next time interval (t)
    t = pf_df.index[i]
    price = price_df.iloc[i]
    pnl = pnl_df.iloc[i]

    # Mark-to-market the portfolio 
    pnl = pf * price
    nav = pnl.sum()

    # Calc latest portfolio weights based on NAV
    curr_weight = pnl / nav
     
    # Calc delta of current to target weight
    target_weight = weight_df.iloc[i]
    delta_weight = target_weight - curr_weight

    # Based on buffer decide if trade should be made
    do_trade = delta_weight > trade_buffer
    do_trade['cash'] = False
    if do_trade.all():
        pf_df.iloc[i] = pf
        continue

    # Assume min fixed commission so trade to target weight
    adj_target_weight = target_weight
    adj_delta_weight = adj_target_weight - curr_weight

    # Calc trade to achieve adjusted target weight
    trade_value = adj_delta_weight * pnl
    trade_size = trade_value / price

    # Calc commission
    #fee = min_commission(price, trade_size)

    post_trade_pf = pf.copy()
    post_trade_pf = pf + trade_size
    #post_trade_pf['cash'] -= fee
    #post_trade_pf['cash'] -= trade_value.sum()
    #post_trade_exposure = post_trade_pf * price

    #adj_target_value = adj_target_weight * nav
    #adj_target_size = adj_target_value / price
    pf_df.iloc[i, do_trade] = post_trade_pf[do_trade]
    

    # Append data for this time interval to the result 
    series = pd.concat(
        [price, pf, pnl, curr_weight, target_weight, delta_weight, do_trade, 
            adj_target_weight, adj_delta_weight, trade_value, trade_size, post_trade_pf], 
        keys=['price', 'pf', 'pnl', 'curr_weight', 'target_weight', 'delta_weight', 'do_trade', 
            'adj_target_weight', 'adj_delta_weight', 'trade_value', 'trade_size', 
            'post_trade_pf'],
        axis=1)
    series['datetime'] = t
    series = series.set_index(['datetime', series.index])
    result = pd.concat([result, series])


display(result)


KeyboardInterrupt: 